# CSV processing for Redezeit's dashboard scraped data

This notebook's purpose is to process the scraped csv files from Redezeit's Looker Studio dashboard and rename the csv files with a more meaningful name.
We preferred a nb instead of a .py file to have a controlled view of each line of code.

In [ ]:
import pandas as pd
import os
from pathlib import Path
from datetime import datetime


### Path, file list, validation

In [ ]:
#  Current nb location
BASE_DIR = BASE_DIR = os.getcwd()

#  Path to raw data folder
folder_path = os.path.normpath(os.path.join(BASE_DIR, '..', 'data', 'raw'))

#  Create output folder for cleaned CSV'set
clean_folder = os.path.join(BASE_DIR, '..', 'data', 'clean')
os.makedirs(clean_folder, exist_ok=True)

file_names = [
    'landingpage.csv',
    'user_behaviors.csv',
    'what_devices_used_chart.csv',
    'what_did_user_do.csv',
    'where_did_they_come_from.csv',
    'where_new_visitors_come_from_chart.csv',
    'who_was_visiting_chart.csv'
]


# Standardize final filenames

final_names = {
    'landingpage.csv':                        'landing_page_views.csv',
    'user_behaviors.csv':                     'user_sessions.csv',
    'what_devices_used_chart.csv':            'device_usage.csv',
    'what_did_user_do.csv':                   'user_events.csv',
    'where_did_they_come_from.csv':           'traffic_sources.csv',
    'where_new_visitors_come_from_chart.csv': 'traffic_source_chart.csv',
    'who_was_visiting_chart.csv':             'daily_visitors_chart.csv'
}

#  Expected numer of cols in each file (for validation)
expected_columns = {
    'landingpage.csv':                        3,
    'user_behaviors.csv':                     3,
    'what_devices_used_chart.csv':            4,
    'what_did_user_do.csv':                   6,
    'where_did_they_come_from.csv':           3,
    'where_new_visitors_come_from_chart.csv': 6,
    'who_was_visiting_chart.csv':             6
}


####  Helper: sort by week
##### Not ready yet, overwritting everything atm bc i need then ratbros (advice bros)

In [ ]:
'''
def extract_week_folder(path: str) -> str:
    """
    Extracts the ISO calendar week folder name (e.g. '2025_KW07') based on the first valid
    date found in the 'Datum' column of a CSV file.

    Args:
        path (str): Path to the CSV file.

    Returns:
        str: Folder name representing year and calendar week, or "unknown_week" if not found.
    """
    try:
        with open(path, encoding="utf-8") as f:
            # Read header line and remove Byte Order Mark (BOM) if present
            header_line = f.readline().lstrip('\ufeff')
            header = header_line.strip().split(';')
            
            # Convert header names to lowercase for case-insensitive search
            header_lower = [h.lower() for h in header]
            
            # Find the index of the 'datum' column (date column)
            datum_idx = header_lower.index("datum")
            
            # Iterate through each line in the file to find a valid date
            for line in f:
                columns = line.strip().split(';')
                
                # Skip line if it doesn't have enough columns
                if len(columns) <= datum_idx:
                    continue
                
                date_str = columns[datum_idx].strip()
                
                try:
                    # Parse the date string assuming format YYYY-MM-DD
                    dt = datetime.strptime(date_str, '%Y-%m-%d')
                    
                    # Extract year and ISO calendar week number
                    year, week, _ = dt.isocalendar()
                    
                    # Format the folder name like '2025_KW07'
                    week_folder = f"{year}_KW{week:02d}"
                    
                    # Confirm success once, then return the folder name
                    print(f"✅ Week folder extracted: {week_folder} from file {path}")
                    return week_folder
                
                except ValueError:
                    # If date parsing fails, try next line
                    continue
                    
    except FileNotFoundError:
        print(f"⚠️ File not found: {path}")
    except Valu
'''

####  Helper: column count validation

In [ ]:
def validate_csv(path: str, expected_cols: int):
    """
    Scan a semicolon-delimited CSV and report any lines
    that don’t have exactly expected_cols fields.
    Returns a list of (line_number, text) for the bad lines.
    """
    issues = []
    with open(path, encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            if line.count(";") + 1 != expected_cols:
                issues.append((i, line.rstrip("\n")))
    return issues

### Morphin'-zeit

In [ ]:
cleaned_files = []

for raw_fname in file_names:
    raw_path = os.path.join(folder_path, raw_fname)
    final_name = final_names.get(raw_fname, raw_fname)

    # Ensure destination folder exists
    os.makedirs(clean_folder, exist_ok=True)
    clean_path = os.path.join(clean_folder, final_name)

    # Copy raw file to clean folder
    with open(raw_path, 'r', encoding='utf-8') as infile, \
         open(clean_path, 'w', encoding='utf-8') as outfile:
        outfile.write(infile.read())

    # Validate cleaned file columns count
    exp = expected_columns.get(final_name)
    if exp:
        problems = validate_csv(clean_path, exp)
        if problems:
            print(f"⚠️ {final_name} has {len(problems)} malformed lines:")
            for ln, txt in problems[:5]:
                print(f"  line {ln}: {txt}")

    cleaned_files.append(clean_path)

print("✅ All files copied, renamed, and validated in clean folder.")


### 5) Load CSV's into df

By loading everything in df's, we make sure things are smooth and working, ready to load into the DB. Converting types here makes sense because this way we can catch errors before we load everything in Postgres

In [ ]:
dataframes = {}
for root, dirs, files in os.walk(clean_folder):
    for fname in files:
        if fname.endswith(".csv"):
            path = os.path.join(root, fname)
            key = final_names.get(fname, fname.replace(".csv", ""))
            df = pd.read_csv(path, sep=';', encoding='utf-8')
            dataframes[key] = df
            print(f"✅{key} loaded from {path}: {df.shape}")

### 6)  Cleaning function

In [ ]:
def clean_and_cast_columns(df):
    """
    Cleans and casts DataFrame columns:
    - Dates to datetime
    - Durations to timedelta strings and numeric seconds/days
    - Percentages to floats
    - Numeric columns cast to int if possible, else float
    - Preserve strings as is

    Returns errors dict if any issues, else None
    """

    errors = {}

    for col in df.columns:
        is_obj = df[col].dtype == "object"
        sample = df[col].dropna().astype(str) if is_obj else None

        # 1) Date conversion
        if "datum" in col.lower():
            converted = pd.to_datetime(df[col], errors="coerce")
            failed = converted.isna() & df[col].notna()
            if failed.any():
                errors[col] = {'datetime_failures': failed.sum()}
            df[col] = converted
            continue

        # 2) Duration hh:mm:ss
        if is_obj and sample.str.match(r"^\d{2}:\d{2}:\d{2}$").all():
            td = pd.to_timedelta(df[col], errors="coerce")
            failed = td.isna() & df[col].notna()
            if failed.any():
                errors[col] = {'timedelta_failures': failed.sum()}
            df[col] = td.astype(str).str[-8:]
            df[f"{col}_seconds"] = td.dt.total_seconds()
            df[f"{col}_days"] = td.dt.total_seconds() / 86400
            continue

        # 3) Percentage conversion
        if is_obj and sample.str.match(r"^[\d\.\,]+\s*%$").all():
            stripped = sample.str.replace("%", "", regex=False).str.replace(",", ".", regex=False)
            converted = pd.to_numeric(stripped, errors="coerce") / 100
            failed = converted.isna() & df[col].notna()
            if failed.any():
                errors[col] = {'percentage_failures': failed.sum()}
            df[col] = converted
            continue

        # 4) Numeric casting for numeric columns or numeric-like strings
        if not is_obj:
            # Numeric column - check if all integers (no decimals)
            if pd.api.types.is_integer_dtype(df[col]):
                # Already int, no action needed
                continue
            elif pd.api.types.is_float_dtype(df[col]):
                # Check if float column can be converted to int safely (no decimals)
                if (df[col].dropna() % 1 == 0).all():
                    df[col] = df[col].astype('Int64')  # nullable integer dtype to keep NaNs
                # else keep as float
                continue
            else:
                # Other numeric types, convert to float by default
                df[col] = df[col].astype(float)
                continue

        # Try converting object columns with numeric values
        if is_obj:
            # Check if numeric-like strings
            if sample.str.match(r"^\d+(\.\d+)?$").all():
                converted = pd.to_numeric(df[col], errors="coerce")
                failed = converted.isna() & df[col].notna()
                if failed.any():
                    errors[col] = {'numeric_failures': failed.sum()}
                    df[col] = df[col].astype(str)  # fallback keep as string
                else:
                    # Cast to int if all whole numbers else float
                    if (converted.dropna() % 1 == 0).all():
                        df[col] = converted.astype('Int64')
                    else:
                        df[col] = converted.astype(float)
                continue

            # Otherwise preserve as string
            df[col] = sample

    if errors:
        return df, errors
    else:
        print("✅ All columns cleaned and cast successfully.")
        return df, None


### 7) Run them Jewells!

In [ ]:
for name, df in dataframes.items():
    print(f"🧙🏽‍♂️ ...morphin' DataFrame: '{name}'")
    cleaned_df, error_info = clean_and_cast_columns(df)
    dataframes[name] = cleaned_df
    if error_info:
        print(f"Errors in '{name}': {error_info}")

### 8) Datatype check for each df

In [ ]:
for name, df in dataframes.items():
    print(f"'{name}' data types:")
    print(df.dtypes)
    print("-" * 40)

### 9) Head check for each df

In [ ]:
for name, df in dataframes.items():
    print(f"'{name}' head:")
    print(df.head())
    print("-" * 60)